In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
# Defining class labels
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [7]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [8]:
# Normalizing the data

x_train_scaled = x_train/255
x_test_scaled = x_test/255

In [9]:

y_train_categorical = keras.utils.to_categorical(y_train,num_classes=10, dtype="float32")

y_test_categorical = keras.utils.to_categorical(y_test,num_classes=10, dtype="float32")

In [10]:
def get_model():
    model = keras.Sequential([  # Defining a Sequential model.
        keras.layers.Flatten(input_shape=(32, 32, 3)),  # Flattening input data.
        keras.layers.Dense(3000, activation="relu"),  # Adding a Dense layer with ReLU activation.
        keras.layers.Dense(1000, activation="relu"),  # Adding another Dense layer with ReLU activation.
        keras.layers.Dense(10, activation="sigmoid")  # Adding a Dense output layer with sigmoid activation.
    ])

    model.compile(optimizer="SGD",  # Compiling the model with Stochastic Gradient Descent optimizer.
                  loss="categorical_crossentropy",  # Using categorical cross-entropy loss function.
                  metrics=["accuracy"])  # Tracking accuracy during training.
    return model

**The x_train_scaled & x_test_scaled are numpy array & we want convert this to tensorflow dataset becoz tensorflow dataset support batching. & this process is called batching**

**In the data is to big & we want to read it step by step then we use data pipeline, without using this, it'll read in one shot & it might not fit.**

In [11]:
# Creating a TensorFlow Dataset

train_tf_dataset = tf.data.Dataset.from_tensor_slices((x_train_scaled,y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((x_test_scaled,y_test_categorical))

2024-05-26 17:20:38.029813: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-26 17:20:38.031891: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-05-26 17:20:38.031911: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-05-26 17:20:38.032060: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-26 17:20:38.032926: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
type(train_tf_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

**tf.distribute.MirroredStrategy is a TensorFlow distribution strategy that supports synchronous training on multiple GPUs. It mirrors (or replicates) all of your model's variables across the available devices. This means that each device maintains its own copy of the model and gradients are aggregated across all devices at each step.**

In [14]:
# Setting up a MirroredStrategy for distributed training
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [15]:
strategy.num_replicas_in_sync

1

In [19]:
# Setting batch size per replica
BATCH_SIZE_PER_REPLICA = 250

# Calculating batch size
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

# Creating a batched and prefetched training dataset
train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Creating a batched test dataset
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [22]:
%%timeit -n1 -r1
# Creating a distributed training scope
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset,epochs=50)

2024-05-26 18:28:04.918533: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 10
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
expe

Epoch 1/50


2024-05-26 18:28:05.124519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-26 18:28:05.299086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-26 18:28:05.304478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  1/200 [..............................] - ETA: 1:07 - loss: 2.5966 - accuracy: 0.1240

2024-05-26 18:28:05.494015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

200/200 [==============================] - 6s 30ms/step - loss: 9.7466 - accuracy: 0.2284
Epoch 49/50
  6/200 [..............................] - ETA: 4s - loss: 9.2346 - accuracy: 0.2480

2024-05-26 18:33:10.811847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-26 18:33:10.817683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 7s 36ms/step - loss: 9.9464 - accuracy: 0.2273
Epoch 50/50
  5/200 [..............................] - ETA: 6s - loss: 9.4189 - accuracy: 0.2600 

2024-05-26 18:33:18.331175: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-26 18:33:18.337037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 6s 30ms/step - loss: 10.2179 - accuracy: 0.2286
5min 19s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [23]:
gpu_model.evaluate(test_dataset)

2024-05-26 18:34:05.768601: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 10000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:1"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 10
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
expe

 4/40 [==>...........................] - ETA: 0s - loss: 1.9157 - accuracy: 0.3100 

2024-05-26 18:34:06.015851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - 1s 22ms/step - loss: 1.8964 - accuracy: 0.3196


[1.896438479423523, 0.319599986076355]